# Importing Libraries

In [1]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as soup
import re
import pandas as pd
import numpy as np

In [2]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk import pos_tag
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('wordnet')
from nltk.corpus import wordnet
nltk.download('averaged_perceptron_tagger')
from textblob import TextBlob

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vicky\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vicky\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vicky\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\vicky\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


# Importing Input

In [3]:
data=pd.read_excel(r'C:\Users\vicky\Downloads\Input.xlsx')
data.head(10)

,URL_ID,URL
0,1,https://insights.blackcoffer.com/how-is-login-...
1,2,https://insights.blackcoffer.com/how-does-ai-h...
2,3,https://insights.blackcoffer.com/ai-and-its-im...
3,4,https://insights.blackcoffer.com/how-do-deep-l...
4,5,https://insights.blackcoffer.com/how-artificia...
5,6,https://insights.blackcoffer.com/how-are-genet...
6,7,https://insights.blackcoffer.com/how-is-ai-use...
7,8,https://insights.blackcoffer.com/benefits-of-b...
8,9,https://insights.blackcoffer.com/how-big-data-...
9,10,https://insights.blackcoffer.com/how-will-ai-m...


# Simple Web Scraping Using Beautiful Soup

In [4]:
data['TITLE']=""
data['CONTENT']=""
for i in range(170):
    url = data['URL'][i]
    req = Request(url , headers={'User-Agent': 'Chrome/5.0'})

    webpage = urlopen(req).read()
    page_soup = soup(webpage, "html.parser")
    title=page_soup.find("h1",{"class":"entry-title"})
    data['TITLE'][i]=title.get_text()
    content=page_soup.find("div",{"class":"td-post-content"})
    data['CONTENT'][i]=content.get_text()

<ipython-input-4-dfbf3fe5a399>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['TITLE'][i]=title.get_text()
<ipython-input-4-dfbf3fe5a399>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['CONTENT'][i]=content.get_text()


Combining Title and Content Columns. Removing new lines and some other unnecessary characters.

In [5]:
for i in range(170):
    data['CONTENT'][i]=data['TITLE'][i]+data['CONTENT'][i]
    data['CONTENT'][i]=data['CONTENT'][i].replace("\n", ". ")
    data['CONTENT'][i]=data['CONTENT'][i].replace("\xa0.", " ")
data=data.drop('TITLE',axis=1)

<ipython-input-5-ce57017fac5a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['CONTENT'][i]=data['TITLE'][i]+data['CONTENT'][i]
<ipython-input-5-ce57017fac5a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['CONTENT'][i]=data['CONTENT'][i].replace("\n", ". ")
<ipython-input-5-ce57017fac5a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['CONTENT'][i]=data['CONTENT'][i].replace("\xa0.", " ")


Removing any extra Periods

In [6]:
for j in range(170):
    n=""
    h=False
    for i in range(len(data['CONTENT'][j])):
        if (h==False and data['CONTENT'][j][i]=='.'):
            n+=data['CONTENT'][j][i]
            h=True
        elif (h==True and (data['CONTENT'][j][i]=='.' or data['CONTENT'][j][i]==' ')):
            h=True
        elif (h==True):
            n+=" "
            h=False
        if (h==False):
            n+=data['CONTENT'][j][i]
    data['CONTENT'][j]=n

<ipython-input-6-0efdebbc6c8b>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['CONTENT'][j]=n


# Cleaning and Lemmatization

In [7]:
def clean(text):
# Removes all special characters and numericals leaving the alphabets
    text = re.sub('[^A-Za-z]+', ' ', text)
    return text

# Cleaning the text in the review column
data['CleanedContent'] = data['CONTENT'].apply(clean)

In [8]:
data['POStagged']=""
pos_dict = {'J':wordnet.ADJ, 'V':wordnet.VERB, 'N':wordnet.NOUN, 'R':wordnet.ADV}
def token_stop_pos(text):
    tags = pos_tag(word_tokenize(text))
    newlist = []
    for word, tag in tags:
        if word.lower() not in set(stopwords.words('english')):
            newlist.append(tuple([word, pos_dict.get(tag[0])]))
    return newlist

data['POStagged'] = data['CleanedContent'].apply(token_stop_pos)

In [9]:
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
def lemmatize(pos_data):
    lemma_rew = " "
    for word, pos in pos_data:
        if not pos:
            lemma = word
            lemma_rew = lemma_rew + " " + lemma
        else:
            lemma = wordnet_lemmatizer.lemmatize(word, pos=pos)
            lemma_rew = lemma_rew + " " + lemma
    return lemma_rew

data['Lemma'] = data['POStagged'].apply(lemmatize)

# Sentiment Cloumns

In [10]:
data['PositiveWords']=0
data['NegativeWords']=0
data['Polarity']=0
data['Subjectivity']=0
for j in range(170):
    List=word_tokenize(data['Lemma'][j])
    Positive=0
    Negative=0
    for i in range(len(List)):
        if (TextBlob(List[i]).sentiment.polarity>0):
            Positive+=1
        elif (TextBlob(List[i]).sentiment.polarity<0):
            Negative+=1
    data.loc[j, 'PositiveWords']=Positive
    data.loc[j, 'NegativeWords']=Negative
    data.loc[j, 'Polarity'] = (Positive-Negative)/(Positive+Negative+0.000001)
    data.loc[j, 'Subjectivity'] = (Positive+Negative)/(len(List)+0.000001)

# Remaining Columns

In [11]:
for i in range(170):
    sents = data['CONTENT'][i].split('.')
    avg_len = sum(len(x.split()) for x in sents) / len(sents)
    data.loc[i, 'AVG SENTENCE LENGTH']=avg_len
    data.loc[i, 'AVG NUMBER OF WORDS PER SENTENCE']=avg_len

In [12]:
def syllable_count(word):
    word = word.lower()
    count = 0
    vowels = "aeiouy"
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)):
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
    if word.endswith("e"):
        count -= 1
    if count == 0:
        count += 1
    return count

In [13]:
data['PERCENTAGE OF COMPLEX WORDS']=0
data['FOG INDEX']=0
data['COMPLEX WORD COUNT']=0
data['WORD COUNT']=0
data['SYLLABLE PER WORD']=0
data['PERSONAL PRONOUNS']=0
for j in range(170):
    totalSyllableCount=0
    characters=0
    count=0
    ccount=0
    List=word_tokenize(data['Lemma'][j])
    for i in range(len(List)):
        characters+=len(List[i])
        n=syllable_count(List[i])
        totalSyllableCount+=n
        if (n>2):
            ccount+=1
        count+=1
    data.loc[j, 'PERCENTAGE OF COMPLEX WORDS']=ccount/count
    data.loc[j, 'FOG INDEX']=0.4*(data['AVG SENTENCE LENGTH'][j]+data['PERCENTAGE OF COMPLEX WORDS'][j])
    data.loc[j, 'COMPLEX WORD COUNT']=ccount
    data.loc[j, 'WORD COUNT']=count
    data.loc[j, 'SYLLABLE PER WORD']=totalSyllableCount/len(List)   
    data.loc[j, 'AVG WORD LENGTH']=characters/count

In [14]:
for j in range(170):
    personalPronoun=0
    List=word_tokenize(data['CONTENT'][j])
    for i in range(len(List)):
        if ((List[i].lower()=='i' or List[i].lower()=='we' or List[i].lower()=='us' or List[i].lower()=='my' or List[i].lower()=='ours') and List[i]!='US'):
            personalPronoun+=1
    data.loc[j, 'PERSONAL PRONOUNS']=personalPronoun

In [15]:
data=data.rename(columns={'PositiveWords':'POSITIVE SCORE','NegativeWords':'NEGATIVE SCORE','Polarity':'POLARITY SCORE','Subjectivity':'SUBJECTIVITY SCORE'})

In [16]:
data=data.drop(['CONTENT','CleanedContent','POStagged','Lemma'],axis=1)

In [17]:
data.head()

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,AVG NUMBER OF WORDS PER SENTENCE,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,1,https://insights.blackcoffer.com/how-is-login-...,23,10,0.393939,0.076212,20.222222,20.222222,0.226328,8.179420,98,433,1.939954,4,6.143187
1,2,https://insights.blackcoffer.com/how-does-ai-h...,28,4,0.750000,0.082474,20.062500,20.062500,0.283505,8.138402,110,388,2.095361,2,6.458763
2,3,https://insights.blackcoffer.com/ai-and-its-im...,62,31,0.333333,0.084316,19.287234,19.287234,0.293744,7.832391,324,1103,2.097915,13,6.413418
3,4,https://insights.blackcoffer.com/how-do-deep-l...,12,10,0.090909,0.083650,26.411765,26.411765,0.300380,10.684858,79,263,2.152091,1,6.673004
4,5,https://insights.blackcoffer.com/how-artificia...,35,23,0.206897,0.081690,17.956522,17.956522,0.294366,7.300355,209,710,2.043662,27,6.242254


# Incase you want to export

In [18]:
##data.to_csv('COntentOooonly.csv')